In [ ]:
from pprint import pprint
import datetime
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import pendulum
from sqlalchemy import create_engine

nowtz = pendulum.now('America/Sao_Paulo')

now = nowtz.subtract(days=1)
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
time = now.strftime("%H:%M")
today = now.strftime('%Y-%m-%d')

In [12]:
html = urlopen('https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp?Data=05/04/2022&Data1=20220405&slcTaxa=PRE')
bs = BeautifulSoup(html, 'html.parser')
linhas = bs.find_all('td')

### Criando arrays para guardar os registros da coleta


In [ ]:
dia, idx_252, idx_360 = [],[],[]

## busca e coleta os textos das tags filhas da tabela 
# Como todos os registros estão em um único array, tive que fazer um salto
# de 3 em 3 registros para gravar nos arrays criados
# ##
children = linhas

row = 0
while row < len(children):
    dia.append(children[row].text)
    row = row + 3

row = 1
while row < len(children):
    idx_252.append(children[row].text)
    row = row + 3

row = 2
while row < len(children):
    idx_360.append(children[row].text)
    row = row + 3    
    

In [15]:
df = pd.DataFrame({'Data':'2022-04-05', 'Dia': dia, 'ind252':idx_252, 'ind360':idx_360})

### Transformando o df para colunas em valores

In [16]:
df['ind252'] = df['ind252'].replace(to_replace=',', value='.', regex=True)
df['ind360'] = df['ind360'].replace(to_replace=',', value='.', regex=True)

In [18]:
df['Data'] = df['Data'].astype('datetime64[ns]')
df['Dia'] = df['Dia'].astype(int)
df['ind252'] = df['ind252'].astype(float)
df['ind360'] = df['ind360'].astype(float)

In [20]:
engine = create_engine(f'mssql+pyodbc://{ConfigDB.dbuser()}:{ConfigDB.dbpass()}@{ConfigDB.dbhost()}:1433/{ConfigDB.dbname()}?driver=ODBC+Driver+17+for+SQL+Server')

df.to_sql('web_curva_cdi_b3', engine, index=False, if_exists='append' )
